In [1]:
import os
os.chdir('/home/blueconet/Downloads/Image Classification/src')

In [2]:
from cifar_input import cifar10_input
import ResNet_cifar10_3 as model
import tensorflow as tf

BATCH_SIZE=16
EVAL_SIZE=1000
NUM_GPUS=8

def tower_loss(batch, scope, keep_prob, reuse):
    images, labels = batch
    
    logits = model.inference(images, keep_prob, True, reuse)
    _ = model.loss(logits, labels)
    
    losses = tf.get_collection('losses', scope)
    
    total_loss = tf.add_n(losses, name='total_loss')
    return total_loss

def average_grads(list_grads):
    grads = []
    for i in range(len(list_grads[0])):
        if list_grads[0][i][0] is None:
            grads.append((None, list_grads[0][i][1]))
        else:
            grads.append((tf.reduce_mean([list_grads[j][i][0] for j in range(len(list_grads))],[0]),
                         list_grads[0][i][1]))
    return grads

In [3]:
# Test for learning rate
with tf.Graph().as_default(), tf.device("/cpu:0"):
    train_data, test_data = cifar10_input()
    
    x, y_ = train_data.dist_batch(BATCH_SIZE,20000)
    test_batch = test_data.batch(200)

    keep_prob = tf.placeholder(tf.float32)
    lr = tf.placeholder(tf.float32)
    opt = tf.train.GradientDescentOptimizer(lr)

    with tf.device('/gpu:0'):
        logit = model.inference(x, keep_prob, True)
        loss = model.loss(logit, y_)
        grads = opt.compute_gradients(loss)    
    
    train_op = opt.apply_gradients(grads)
    sess = tf.Session()
    
    rate = 0.1
    
    sess.run(tf.initialize_all_variables())
    tf.train.start_queue_runners(sess=sess)
    for i in range(20):
        _, cross_entropy = sess.run([train_op, loss], feed_dict = {keep_prob: 0.5, lr:rate})
        print("step %d, cross_entropy %g"%(i, cross_entropy))

KeyboardInterrupt: 

In [4]:
# Train in single GPU
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob, false)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)

        with tf.device('/gpu:0'):
            with tf.name_scope('%s_%d' % ('tower', 0)) as scope:
                loss = tower_loss(train_batch, scope)
                grads = opt.compute_gradients(loss)    
    
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.1
        mean_loss = 0.
        least = 10.
        count = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean_loss = mean_loss + cross_entropy/50
    
            if i%50 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, mean_loss))
      
                if mean_loss > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean_loss
                mean_loss=0
        
            if i%250 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                print("test accuracy %g"%(eval_accuracy))

step 50, cross_entropy nan
step 100, cross_entropy nan
step 150, cross_entropy nan
step 200, cross_entropy nan
step 250, cross_entropy nan
test accuracy 0
step 300, cross_entropy nan


KeyboardInterrupt: 

In [3]:
# Train in multi GPU
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)
        
        keep_prob = tf.placeholder(tf.float32)
        
        logit = model.inference(images, keep_prob, False, False)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        tower_grads = []
        tower_losses = []
        
        lr = tf.placeholder(tf.float32)
        opt = tf.train.GradientDescentOptimizer(lr)
        
        for i in range(NUM_GPUS):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                    loss = tower_loss(train_batch, scope, keep_prob, True)
                    grads = opt.compute_gradients(loss)
                    tower_losses.append(loss)
                    tower_grads.append(grads)
                    
        grads = average_grads(tower_grads)
        mean_loss = tf.reduce_mean(tower_losses)
       
        train_op = opt.apply_gradients(grads)
    
        sess = tf.Session()
        
        rate = 0.1
        mean = 0.
        highest = 0.
        
                
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)

In [ ]:
rate = 0.1
with tf.device('/cpu:0'):
    for i in range(20000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

In [8]:
rate = 0.1
with tf.device('/cpu:0'):
    for i in range(10000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 0.0948039
test accuracy 0.784
step 399, cross_entropy 0.734466
test accuracy 0.755
step 599, cross_entropy 0.354488
test accuracy 0.784
step 799, cross_entropy 0.791492
test accuracy 0.814
step 999, cross_entropy 0.297589
test accuracy 0.785
step 1199, cross_entropy 0.391419
test accuracy 0.771
step 1399, cross_entropy 0.164679
test accuracy 0.801
step 1599, cross_entropy 0.289679
test accuracy 0.793
step 1799, cross_entropy 0.287635
test accuracy 0.772
step 1999, cross_entropy 0.26414
test accuracy 0.787
step 2199, cross_entropy 0.444175
test accuracy 0.767
step 2399, cross_entropy 0.236612
test accuracy 0.789
step 2599, cross_entropy 0.402675
test accuracy 0.8
step 2799, cross_entropy 0.158675
test accuracy 0.779
step 2999, cross_entropy 0.330457
test accuracy 0.81
step 3199, cross_entropy 0.975576
test accuracy 0.803
step 3399, cross_entropy 0.462285
test accuracy 0.793
step 3599, cross_entropy 0.154716
test accuracy 0.771
step 3799, cross_entropy 0.255983
te

In [9]:
tf.GraphKeys.UPDATE_OPS

'update_ops'

In [17]:
[len(list_grads[i]) for i in range(len(list_grads))]

[41, 41, 41, 41, 41, 41, 41, 41]

In [4]:
tower_grads[0][26]

(None, <tensorflow.python.ops.variables.Variable at 0x7fa5fd845490>)

In [15]:
[(tf.reduce_mean([list_grads[j][i][0] for j in range(len(list_grads))],[0]),
             list_grads[0][i][1]) for i in range(len(list_grads[0]))]

TypeError: Expected binary or unicode string, got None